In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
df_product = pd.read_csv('03(heart열ID열추가)건강기능식품 품목제조 신고사항 현황_건강기능식품 종류 및 효능 - Copy.csv', encoding='utf8', index_col=0).fillna('항목없음')
df_product.head(2)

,itemID,허가번호,제품명,제조회사,제품타입,제품분류,유통기한,효능,복용법,주의사항,어린이 기호식품,heart
0,0,2.004002e+10,해피트루 포스트 바이오틱스,(주)알피바이오,"고유의 향미가 있고 이미, 이취가 없는 진한 노랑색 알갱이가 있는 하양색 분말","프로바이오틱스, 프락토올리고당, 아연, 비오틴",제조일로부터 18개월,[프락토올리고당] ①장내 유익균 증식 및 배변활동 원활에 도움을 줄 수 있음\n\n...,"1일 1회, 1회 1포를 충분한 물과 함께 섭취하십시오.","[프락토올리고당]\n (가) 섭취 시 가스참, 트림, 복통, 복부팽만감 등이 발생할...",항목없음,2
1,1,2.015001e+10,클린차전자피분말,푸른바이오에스,"이미, 이취가 없고 고유의 향미가 있는 밝은 회황색의 분말",차전자피식이섬유,제조일로부터 24개월까지,[차전자피식이섬유] 혈중 콜레스테롤 개선·배변활동 원활에 도움을 줄 수 있음,건강기능식품 원료로 사용,반드시 충분한 물과 함께 섭취할 것 (액상제외),항목없음,2


In [4]:
# 문구 dense 작업 진행 // 제품분류는 count로, 효능은 tf-idf로 진행
# 하기 내용은 고정이므로 추가 함수작업 없이 진행

corpus_product_category = []
corpus_product_effect = []

for i in range(len(df_product)):
    doc_temp = ''
    doc_temp = df_product['제품분류'].iloc[i]
    corpus_product_category.append(doc_temp)


for i in range(len(df_product)):
    doc_temp = ''
    doc_temp = df_product['효능'].iloc[i]
    corpus_product_effect.append(doc_temp)
    
    
count_vectorizer = CountVectorizer()
tfidf_vectorizer = TfidfVectorizer()
count_product_category = count_vectorizer.fit_transform(corpus_product_category).todense()
tfidf_product_effect = tfidf_vectorizer.fit_transform(corpus_product_category).todense()

In [7]:
## 제품에 따른 위치 확인 필요(index 문제로 iloc으로 찾아야)
## product nam은 인풋 항목으로 찾아야

def find_index(name):
    product_name_no = ''
    for i in range(len(df_product)):
        if product_name == df_product['제품명'].iloc[i]:
            product_name_no = i
    
    return product_name_no

In [2]:
# 코사인 유사도 진행

def cosine_similarity_top5(product_name_no):
    list_cos_product_category = []
    list_cos_product_category = cosine_similarity(count_product_category[product_name_no], count_product_category)
    
    list_cos_product_effect = []
    list_cos_product_effect = cosine_similarity(tfidf_product_effect[product_name_no], tfidf_product_effect)
    
    # 총점 구하기
    
    total_product_cos_score = []

    for i in range(len(list_cos_product_category[0])):
        total_score = ((list_cos_product_category[0][i]) + (list_cos_product_effect[0][i])) / 2
        total_product_cos_score.append(total_score)
    
    # 총점 구한 결과로 제품명과 함께 신규 dataframe 구축 // 작업 중 오류가 있어 list -> dict -> dataframe 으로 변환 작업 진행
    
    list_total_product_cos_score = np.array(total_product_cos_score).T.tolist()
    
    list_product = []

    for i in range(len(df_product)):
        list_product.append(df_product['제품명'].iloc[i])
        
    dict_temp = {'제품명' : list_product, '코사인 유사도' : list_total_product_cos_score}
    df_cos_score = pd.DataFrame(dict_temp)
    df_cos_score = df_cos_score.sort_values(by='코사인 유사도', ascending=False).reset_index()
    
    # Top 5 도출 // 첫 데이터프레임 위치, 제품명 출력
    
    list_cos_top5 = []

    for i in range(1, 6):
        list_temp = []
        list_temp.append(df_cos_score['index'][i])
        list_temp.append(df_cos_score['제품명'][i])
        list_cos_top5.append(list_temp)
    
    # Top 5 항목 data 추출
    
    df_top5 = df_product.iloc[[list_cos_top5[0][0], 
                               list_cos_top5[1][0], 
                               list_cos_top5[2][0], 
                               list_cos_top5[3][0], 
                               list_cos_top5[4][0]]]

 
    return df_top5

In [9]:
cosine_similarity_top5(find_index('해피트루 포스트 바이오틱스'))

,itemID,허가번호,제품명,제조회사,제품타입,제품분류,유통기한,효능,복용법,주의사항,어린이 기호식품,heart
7337,7337,2.004002e+10,프리바이오틱스,아미코젠(주),"이미, 이취가 없는 미백색의 분말","프로바이오틱스, 프락토올리고당, 아연",제조일로부터 18개월,①유산균 증식 및 유해균 억제·배변활동 원활에 도움을 줄 수 있음 \n\n① 장내 ...,"1일 1회, 1회 1포를 그대로 섭취하거나 물과 함께 섭취하십시오.",1) 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n2) 알레르기 체질 등은 ...,항목없음,4
626,626,2.004002e+10,하양공쥬 닥터스 신프로프리바이오틱스,(주)유유헬스케어,"고유의 향미가 있고 이미, 이취가 없는 백색의 분말","프락토올리고당, 프로바이오틱스, 아연",제조일로부터 24개월,장내 유익균 증식 및 배변활동 원활에 도움을 줄 수 있음\n\n유산균 증식 및 유해...,"1일 1회, 1회 1포를 충분한 물과 함께 섭취하십시오.","(가) 섭취 시 가스참, 트림, 복통, 복부팽만감 등이 발생할 수 있음\n(나) 이...",항목없음,2
6473,6473,2.020001e+10,몬스터 바이오틱스 100억,농업회사법인 주식회사 내츄럴엔,"이미, 이취가 없고 고유의 향미가 있는 미백색의 분말","프로바이오틱스, 아연, 프락토올리고당",제조일로부터 18개월,① 유산균 증식 및 유해균 억제에 도움을 줄 수 있음 ②배변활동 원활에 도움을 줄 ...,"1일 1회, 1회 1포(5g)를 직접 또는 물과 함께 섭취하십시오.",1. 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n2. 알레르기 체질 등은 ...,항목없음,1
6480,6480,2.004002e+10,이너락토플러스,(주)비오팜,"이미, 이취가 없고 고유의 향미가 있는 백색의 분말","아연, 프로바이오틱스, 프락토올리고당",제조일로부터 18개월,[프락토올리고당] ① 장내 유익균 증식 및 배변활동 원활에 도움을 줄 수 있음\n[...,"1일 1회, 1회 1포를 직접 또는 물과 함께 섭취하십시오.",- 질환이 있거나 의약품 복용 시 전문가와 상담할 것\n- 알레르기 체질 등은 개인...,항목없음,2
6564,6564,2.004002e+10,프로프리 신바이오틱스 생유산균,(주)녹십초알로에,"이미, 이취가 없고 고유의 향미가 있는 백색의 분말","프로바이오틱스, 프락토올리고당, 아연",제조일로부터 18개월,유산균 증식 및 유해균 억제·배변활동 원활에 도움을 줄 수 있음\n\n① 유익균 증...,"1일 1회, 1회 1포를 직접 섭취하거나 물, 음료 등과 같이 섭취하십시오.",(가) 질환이 있거나 의약품 복용 시 전문가와 상담할 것 (나) 알레르기 체질 등은...,항목없음,1


In [3]:
# ===================================================

In [5]:
# 유사도분석 건강기능식품 추천 코드입니다 
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def vectorize_item(df_item):
# 문구 dense 작업 진행 // 제품분류는 count로, 효능은 tf-idf로 진행
# 하기 내용은 고정이므로 추가 함수작업 없이 진행
    corpus_product_category = []
    corpus_product_effect = []

    for i in range(len(df_item)):
        doc_temp = ''
        doc_temp = df_item['제품분류'].iloc[i]
        corpus_product_category.append(doc_temp)


    for i in range(len(df_item)):
        doc_temp = ''
        doc_temp = df_item['효능'].iloc[i]
        corpus_product_effect.append(doc_temp)

    count_vectorizer = CountVectorizer()
    tfidf_vectorizer = TfidfVectorizer()
    
    count_product_category = count_vectorizer.fit_transform(corpus_product_category).todense()
    tfidf_product_effect = tfidf_vectorizer.fit_transform(corpus_product_category).todense()
    return count_product_category,tfidf_product_effect

# 코사인 유사도 진행

def cosine_similarity_top5(count_product_category,tfidf_product_effect, item_name,df_item):
    # get item index
    product_name_no = ''
    for i in range(len(df_item)):
        if item_name == df_item['제품명'].iloc[i]:
            product_name_no = i
    
    list_cos_product_category = []
    list_cos_product_category = cosine_similarity(count_product_category[product_name_no], count_product_category)
    
    list_cos_product_effect = []
    list_cos_product_effect = cosine_similarity(tfidf_product_effect[product_name_no], tfidf_product_effect)
    
    # 총점 구하기
    total_product_cos_score = []
    for i in range(len(list_cos_product_category[0])):
        total_score = ((list_cos_product_category[0][i]) + (list_cos_product_effect[0][i])) / 2
        total_product_cos_score.append(total_score)
    
    # 총점 구한 결과로 제품명과 함께 신규 dataframe 구축 // 작업 중 오류가 있어 list -> dict -> dataframe 으로 변환 작업 진행
    list_total_product_cos_score = np.array(total_product_cos_score).T.tolist()
    
    list_product = []
    for i in range(len(df_item)):
        list_product.append(df_item['제품명'].iloc[i])
    dict_temp = {'제품명' : list_product, '코사인 유사도' : list_total_product_cos_score}
    df_cos_score = pd.DataFrame(dict_temp)
    df_cos_score = df_cos_score.sort_values(by='코사인 유사도', ascending=False).reset_index()
    
    # Top 5 도출 // 첫 데이터프레임 위치, 제품명 출력
    items_top5_list = list(df_cos_score['index'][1:6])
    return items_top5_list


In [10]:
df_item = pd.read_csv("03itemdf.csv")

count_product_category,tfidf_product_effect = vectorize_item(df_item)

result = cosine_similarity_top5(count_product_category,tfidf_product_effect,'프리바이오틱스',df_item)

result

[2142, 5551, 4133, 12902, 20473]

In [15]:
import numpy
count_product_category
count_product_category.dump("count_product_category.dat")
count_product_category = numpy.load("count_product_category.dat")
mat

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [20]:
# count_product_category
# np.savetxt('test1.txt', count_product_category, fmt='%d')
# b = np.loadtxt('test1.txt', dtype=int)
# a==ba

In [7]:
# corpus_product_category = []
# corpus_product_effect = []

# for i in range(len(df_product)):
#     doc_temp = ''
#     doc_temp = df_product['제품분류'].iloc[i]
#     corpus_product_category.append(doc_temp)


# for i in range(len(df_product)):
#     doc_temp = ''
#     doc_temp = df_product['효능'].iloc[i]
#     corpus_product_effect.append(doc_temp)

    
    
# len(corpus_product_category)
# len(corpus_product_effect)

In [8]:
# # 분류의 경우 카테고리이기 때문에 카운트 벡터라이져 사용
# # 효능의 경우 신체기관 및 기타 내용도 함께 들어가 있기 때문에 tf-idf 사용(분류 재료 별 효능이 동일하게 입력되기 때문)

# count_vectorizer = CountVectorizer()
# tfidf_vectorizer = TfidfVectorizer()
# count_product_category = count_vectorizer.fit_transform(corpus_product_category).todense()
# tfidf_product_effect = tfidf_vectorizer.fit_transform(corpus_product_category).todense()


In [9]:
# type(count_product_category)

In [10]:
# ## 제품에 따른 위치 확인 필요(index 문제로 iloc으로 찾아야)
# ## product nam은 인풋 항목으로 찾아야

# product_name = '해피트루 포스트 바이오틱스'
# product_name_no = ''

# for i in range(len(df_product)):
#     if product_name == df_product['제품명'].iloc[i]:
#         product_name_no = i
#         print(i)

In [11]:
# list_cos_product_category = []
# list_cos_product_category = cosine_similarity(count_product_category[product_name_no], count_product_category)

# # for i in range(len(count_product_category)):
# #     list_cos_product_category.append(cosine_similarity(count_product_category[product_name_no], count_product_category[i]).tolist())



In [12]:
# len(list_cos_product_category)

In [13]:
# list_cos_product_category[0:10]

In [14]:
# tfidf_product_effect.shape

In [15]:
# cosine_similarity(tfidf_product_effect[product_name_no], tfidf_product_effect)

In [16]:
# list_cos_product_effect = []
# list_cos_product_effect = cosine_similarity(tfidf_product_effect[product_name_no], tfidf_product_effect)

In [17]:
# list_cos_product_effect[0]

In [18]:
# len(list_cos_product_effect)

In [19]:
# list_cos_product_effect[0]

In [20]:
# # 인풋받은 제품의 제품 분류와 효능의 각각 코사인 유사도를 더하여 최종 값 출력하여 최종 점수 출력

# total_product_cos_score = []

# for i in range(len(list_cos_product_category[0])):
#     total_score = ((list_cos_product_category[0][i]) + (list_cos_product_effect[0][i])) / 2
#     total_product_cos_score.append(total_score)


In [21]:
# total_product_cos_score

In [22]:
# list_total_product_cos_score = np.array(total_product_cos_score).T.tolist()
# list_total_product_cos_score

In [23]:
# list_product = []

# for i in range(len(df_product)):
#     list_product.append(df_product['제품명'].iloc[i])
    
# list_product

In [24]:
# dict_temp = {'제품명' : list_product, '코사인 유사도' : list_total_product_cos_score}
# df_cos_score = pd.DataFrame(dict_temp)
# df_cos_score.head()

In [25]:
# df_cos_score = df_cos_score.sort_values(by='코사인 유사도', ascending=False).reset_index()
# df_cos_score.head(6)

In [26]:
# list_cos_top5 = []

# for i in range(1, 6):
#     list_cos_top5.append(df_cos_score['제품명'][i])
    
# list_cos_top5